In [1]:
import torch
import torchvision
from chatglm1 import *
from chatglm2 import *
import pandas as pd
import json
import time
import faiss
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from Dataset_improved import *
# from chains.local_doc_qa import LocalDocQA
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.docstore.base import AddableMixin, Docstore
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score,precision_recall_curve
# langchain.vectorstores.faiss
#加载全部数据库
vector_data = read_json_file("/mnt/workspace/pangtianqi/medical_kb_chatbot/data/dataset.json")
vector_dataset = MyDataset(vector_data)
vector_dataloader = torch.utils.data.DataLoader(vector_dataset, batch_size=1, shuffle=False)
#加载测试数据库
raw_data = read_json_file("/mnt/workspace/pangtianqi/medical_kb_chatbot/train/m3e_test_finetune.json")
testing_dataset = MyDataset(raw_data)
testing_dataloader = torch.utils.data.DataLoader(testing_dataset, batch_size=1, shuffle=False)

/home/pai/envs/kb_chat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-10-20 03:50:25,806] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_87996/4047337095.py:26 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_87996/4047337095.py'                        │
│                                                                                                  │
│ /mnt/workspace/pangtianqi/medical_kb_chatbot/train/Dataset_improved.py:12 in __init__            │
│                                                                                                  │
│     9 # 自定义Dataset类                                                                          │
│    10 class MyDataset(Dataset):                                                                  │
│    11 │   def __init__(self, raw_data):                                                          │
│ ❱  12 │   │   self.data = data_to_list(raw_data)                                                 │
│    13 │                                                                                          │
│    14 │   def __getitem__(self, idx):                                                            │
│    15 │   │   return self.data[idx]                                                              │
│                                                                                                  │
│ /mnt/workspace/pangtianqi/medical_kb_chatbot/train/Dataset_improved.py:61 in data_to_list        │
│                                                                                                  │
│    58 │   │   │   label_1：对应d1的label                                                         │
│    59 │   """                                                                                    │
│    60 │   df_data = pd.DataFrame(data)                                                           │
│ ❱  61 │   df_data.set_index('问题编号', inplace=True)                                            │
│    62 │   df_data.fillna("-", inplace=True) #把主要成分为null的剔除                              │
│    63 │   data = df_data.replace('', '-')#把 ”“ 替换为”—“                                        │
│    64 │   # data = data.replace('暂无数据', '-')                                                 │
│                                                                                                  │
│ /home/pai/envs/kb_chat/lib/python3.9/site-packages/pandas/core/frame.py:5859 in set_index        │
│                                                                                                  │
│    5856 │   │   │   │   │   │   missing.append(col)                                              │
│    5857 │   │                                                                                    │
│    5858 │   │   if missing:                                                                      │
│ ❱  5859 │   │   │   raise KeyError(f"None of {missing} are in the columns")                      │
│    5860 │   │                                                                                    │
│    5861 │   │   if inplace:                                                                      │
│    5862 │   │   │   frame = self                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: "None of ['问题编号'] are in the columns"

In [2]:
test_data = read_json_file("/mnt/workspace/pangtianqi/medical_kb_chatbot/train/test_finetune.json")

In [6]:
#加载模型
config = ChatGLMConfig()
model = MyMLP(config)
#model =  MyGLU(hidden_size = 768)
device = model.device
model.load_state_dict(torch.load("/mnt/workspace/pangtianqi/medical_kb_chatbot/train/model/best_m3e_chatglm2_L2_v1.ph"))
#model.load_state_dict(torch.load("model/best_chatglm1_v1.ph"))
model = model.to(device)
print(model)
# print(model)
encoder = model.embed_model
dim =768

Embedding size: 768
MyMLP(
  (embed_model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
    (2): Normalize()
  )
  (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (dense_h_to_4h): Linear(in_features=768, out_features=6144, bias=False)
  (dense_4h_to_h): Linear(in_features=3072, out_features=768, bias=False)
)


In [3]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('/mnt/workspace/pangtianqi/medical_kb_chatbot/finetuned_m3e_model', device='cuda:0')

No sentence-transformers model found with name /mnt/workspace/pangtianqi/medical_kb_chatbot/finetuned_m3e_model. Creating a new one with MEAN pooling.


In [4]:
#labels_recall ->所有label存在一个列表
#labels ->按照问题划分的label列表
testing_text = []
labels= []
labels_list_recall = []
for batch_data in test_data:
    # query = test_data[batch_data]["query"]
    data = test_data[batch_data]["content"]
    label = test_data[batch_data]["label"]
    # data, label = batch_data[2:]
    for idx in data:
        for content in idx:
            testing_text.append(content)
    #labels_list_recall.append(label)
    for i in label:
        labels.append(i)
    # print(labels_list)
# labels_recall = [torch.cat(tensor).tolist() for tensor in labels_list_recall]
# labels = [tensor.item() for tensor in labels_list]
#print("testing_text",testing_text)
# print("label_recall",labels_recall)
# print(labels)

In [7]:
#baseline: embedding vector_store
#index ->baseline的索引
content_list = []
for batch_data in testing_text:
    # data, label = batch_data[2:]
    content_vectors = encoder.encode(batch_data)
    content_list.append(content_vectors)

vectors = np.array(content_list,dtype=np.float32)
# print("vectors",len(vectors))
# 选择faiss index
faiss.normalize_L2(vectors)
index = faiss.IndexFlatIP(dim)
# 数据插入
# 训练索引获取重构能力
index.train(vectors)
index.add(vectors)

In [8]:
#model: embedding vector_store
#index_ -> model的索引
content_list_ = []
for batch_data in testing_text:
    # data, label = batch_data[2:]
    content_vectors_ = model(batch_data)
    content_vectors_ = content_vectors_.cpu()
    content_vectors_ = content_vectors_.detach().numpy()
    content_list_.append(content_vectors_)

vectors_ = np.array(content_list_,dtype=np.float32)
# print("vectors_",vectors_.shape)

# 选择faiss index
faiss.normalize_L2(vectors_)


index_ = faiss.IndexFlatIP(dim)

# 数据插入
# 训练索引获取重构能力
index_.train(vectors_)
index_.add(vectors_)

In [8]:
#baseline -> 计算准确率，召回率、精度
k = [2,4,6,8,10,20,30]
for top_k in k:
    print("top_k",top_k)
    count= 0
    precision_all, recall_all, accuracy_all = 0, 0, 0
    start_time = time.time()
    label_pair = []
    D_pair = []
    score_pair = []
    text_list = []
    query_list = []
    len_query = len(test_data)
    hit_count = 0
    for batch_data in test_data:
        count_1 = 0
        tp, fp, fn, tn = 0, 0, 0, 0
        query = test_data[batch_data]["query"]
        data = test_data[batch_data]["content"]
        label = test_data[batch_data]["label"]
        # query = batch_data[1][0]
        # print("query",query)
        # query_list.append(query)
        # data, label = batch_data[2:]
        # print(len(data))
        query_feat = np.array([encoder.encode(query)]) # 得到baseline query embed
        faiss.normalize_L2(query_feat)
        D, I= index.search(query_feat, top_k) #提取出ours的topk
        #print('scores_, indices_',D_, I_[0])
        D_pair.append(D)
        for value in label:
            if value == 1:
                count_1 += 1 #这个q对应的回答中所有label为1的个数
        #print("count_1",count_1)
        # for text in I_[0]:
            # text_list.append(testing_text[text])
            #print("content",testing_text[text])
        for i in I[0]:
            #print("labels[i]",labels[i])
            label_pair.append(labels[i])
        # lst = [] # 模型召回的
        
        # count_0 = len(label[0])
        for i in I[0]:
            if labels[i] == 1:
                tp = tp+1
            # elif labels[i] == 0: # 若labels=1的在模型中的label也等于１
            #     break
        #print("tp",tp)
        #print(len(label))
        if top_k < len(label):
            fp = top_k-tp
        else:
            fp = len(label)-tp
        #print("fp",fp)
        fn = count_1-tp
        #print("fn",fn)
        count_0 = len(labels)-count_1
        tn = count_0-fp
        
        if count_1!=0 : #排除label全为1或全为0的情况
            count+=1
            precision = float(tp/(tp+fp))
            if tp >= count_1:
                recall = 1.0
            else:
                recall = float(tp/(tp+fn))
            accuracy = float((tp+tn)/(tp+fp+tn+fn))

            precision_all += precision
            recall_all += recall
            accuracy_all += accuracy
        if tp!= 0:
            hit_count +=1
        # if count_1 == 0:
        #     len_query -= 1
    print("hit_count",hit_count)
    print("len_query",len_query)
    for item in D_pair:
        for score in item:
            score_pair.extend(score)
    # print("score_pair",len(score_pair_))
    # print("label_pair",len(label_pair_))
    print(f"top_{top_k},recall_ave_m3e: {float(recall_all/count)}")
    print(f"top_{top_k},precision_ave_m3e: {float(precision_all/count)}")
    mAP = average_precision_score(label_pair, score_pair, average='macro')
    print(f"top_{top_k},mAP_m3e:", mAP)
    #print(f"accuracy_ave_m3e: {float(accuracy_all/count)}")
    print(f"top_{top_k},hit_rate_ave_m3e:{float(hit_count/len_query)}")
    end_time = time.time()
    elapsed_time = end_time - start_time
    # print("text_list",text_list)
    print(f"Time spent: {elapsed_time:.5f} seconds")

top_k 2
hit_count 13
len_query 29
top_2,recall_ave_m3e: 0.07224260983603407
top_2,precision_ave_m3e: 0.32
top_2,mAP_m3e: 0.41760874392319564
top_2,hit_rate_ave_m3e:0.4482758620689655
Time spent: 2.20171 seconds
top_k 4
hit_count 24
len_query 29
top_4,recall_ave_m3e: 0.15815896168342036
top_4,precision_ave_m3e: 0.34
top_4,mAP_m3e: 0.41438174541495953
top_4,hit_rate_ave_m3e:0.8275862068965517
Time spent: 2.20209 seconds
top_k 6
hit_count 27
len_query 29
top_6,recall_ave_m3e: 0.2678325333225093
top_6,precision_ave_m3e: 0.34666666666666673
top_6,mAP_m3e: 0.4242507813651208
top_6,hit_rate_ave_m3e:0.9310344827586207
Time spent: 2.30099 seconds
top_k 8
hit_count 27
len_query 29
top_8,recall_ave_m3e: 0.30136105647893935
top_8,precision_ave_m3e: 0.295
top_8,mAP_m3e: 0.3900847514574737
top_8,hit_rate_ave_m3e:0.9310344827586207
Time spent: 2.30069 seconds
top_k 10
hit_count 29
len_query 29
top_10,recall_ave_m3e: 0.3909846186959259
top_10,precision_ave_m3e: 0.292
top_10,mAP_m3e: 0.3662212886730189

In [9]:
#baseline ->计算mAP

# 计算不同阈值下的精确率和召回率
precision, recall, thresholds = precision_recall_curve(label_pair, score_pair)

# 计算每个阈值下的平均精度均值
# average_precision = average_precision_score(true_labels, predicted_probs)
mAP = average_precision_score(label_pair, score_pair, average='macro')
print("mAP_glm1:", mAP)

# 绘制曲线图
plt.plot(recall, precision, color='b', label=f'mAP={mAP:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('m3e_mAP Curve')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_47688/3134912738.py:4 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_47688/3134912738.py'                        │
│                                                                                                  │
│ /home/pai/envs/kb_chat/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:878 in            │
│ precision_recall_curve                                                                           │
│                                                                                                  │
│    875 │   >>> thresholds                                                                        │
│    876 │   array([0.1 , 0.35, 0.4 , 0.8 ])                                                       │
│    877 │   """                                                                                   │
│ ❱  878 │   fps, tps, thresholds = _binary_clf_curve(                                             │
│    879 │   │   y_true, probas_pred, pos_label=pos_label, sample_weight=sample_weight             │
│    880 │   )                                                                                     │
│    881                                                                                           │
│                                                                                                  │
│ /home/pai/envs/kb_chat/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:766 in            │
│ _binary_clf_curve                                                                                │
│                                                                                                  │
│    763 │   │   y_score = y_score[nonzero_weight_mask]                                            │
│    764 │   │   sample_weight = sample_weight[nonzero_weight_mask]                                │
│    765 │                                                                                         │
│ ❱  766 │   pos_label = _check_pos_label_consistency(pos_label, y_true)                           │
│    767 │                                                                                         │
│    768 │   # make y_true a boolean vector                                                        │
│    769 │   y_true = y_true == pos_label                                                          │
│                                                                                                  │
│ /home/pai/envs/kb_chat/lib/python3.9/site-packages/sklearn/metrics/_base.py:243 in               │
│ _check_pos_label_consistency                                                                     │
│                                                                                                  │
│   240 │   │   )                                                                                  │
│   241 │   ):                                                                                     │
│   242 │   │   classes_repr = ", ".join(repr(c) for c in classes)                                 │
│ ❱ 243 │   │   raise ValueError(                                                                  │
│   244 │   │   │   f"y_true takes value in {{{classes_repr}}} and pos_label is not "              │
│   245 │   │   │   "specified: either make y_true take value in {0, 1} or "                       │
│   246 │   │   │   "{-1, 1} or pass pos_label explicitly."                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: y_true takes value in {} and pos_label is not specified: either make y_true take value in {0, 1} or 
{-1, 1} or pass pos_label explicitly.

In [10]:
top_k = 6
# text_list = []
result_m3e = {}  # 创建空字典    
for i,batch_data in enumerate(test_data):
     query = test_data[batch_data]["query"]
     data = test_data[batch_data]["content"]
     label = test_data[batch_data]["label"]
     query_feat = np.array([encoder.encode(query)])
     # query_feat_ = model(query) # 得到model query embed
     # query_feat_ = query_feat_.cpu()
     # query_feat_ = np.array([query_feat_.detach()])
     faiss.normalize_L2(query_feat)
     D, I= index.search(query_feat, top_k) 
     text_list = []
     for text in I[0]:
          text_list.append(testing_text[text])
     # print(text_list)
          # content = testing_text[text]
     result_m3e[i] = {
          "query": query,
          "content": text_list,
          "score": D
     }  
     result_m3e[i]['score'] = result_m3e[i]['score'].tolist()
# print(result_m3e)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_47688/2177126497.py:8 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_47688/2177126497.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'encoder' is not defined

In [11]:
import json
# result[0]['score'] = result[0]['score'].tolist()
# print(result)
# 将字典转换为 JSON 字符串2023年10月6日（星期五）放
json_data_ = json.dumps(result_m3e,ensure_ascii=False)

# 将 JSON 字符串保存到本地文件
file_path = "result_m3e.json"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(json_data_)

In [9]:
#model -> 计算准确率，召回率、精度
k = [2,4,6,8,10,20,30]
for top_k in k:
    print("top_k",top_k)
    count= 0
    precision_all, recall_all, accuracy_all = 0, 0, 0
    start_time = time.time()
    label_pair_ = []
    D_pair_ = []
    score_pair_ = []
    text_list = []
    query_list = []
    len_query = len(test_data)
    hit_count = 0
    for batch_data in test_data:
        count_1 = 0
        tp, fp, fn, tn = 0, 0, 0, 0
        query = test_data[batch_data]["query"]
        data = test_data[batch_data]["content"]
        label = test_data[batch_data]["label"]
        # print("query",query)
        # query_list.append(query)
        # query_feat = np.array([encoder.encode(query)]) # 得到baseline query embed
        query_feat_ = model(query) # 得到model query embed
        query_feat_ = query_feat_.cpu()
        query_feat_ = np.array([query_feat_.detach()])
        faiss.normalize_L2(query_feat_)
        D_, I_ = index_.search(query_feat_, top_k) #提取出ours的topk
        #print('scores_, indices_',D_, I_[0])
        D_pair_.append(D_)
        for value in label:
            if value == 1:
                count_1 += 1 #这个q对应的回答中所有label为1的个数
        #print("count_1",count_1)
        # for text in I_[0]:
            # text_list.append(testing_text[text])
            #print("content",testing_text[text])
        for i in I_[0]:
            #print("labels[i]",labels[i])
            label_pair_.append(labels[i])
        # lst = [] # 模型召回的
        # count_0 = len(label[0])
        for i in I_[0]:
            if labels[i] == 1:
                tp = tp+1
            # elif labels[i] == 0: # 若labels=1的在模型中的label也等于１
            #     break
        #print("tp",tp)
        #print(len(label))
        # fp = top_k-tp
        # print("fp",fp)
        if top_k < len(label):
            fp = top_k-tp
        else:
            fp = len(label)-tp
        #print("fp",fp)
        fn = count_1-tp
        #print("fn",fn)
        count_0 = len(labels)-count_1
        tn = count_0-fp
        if count_1!=0 : #排除label全为1或全为0的情况
            count+=1
            precision = float(tp/(tp+fp))
            accuracy = float((tp+tn)/(tp+fp+tn+fn))
            if tp >= count_1:
                recall = 1.0
            else:
                recall = float(tp/(tp+fn))
            # print("recall",recall)
            precision_all += precision
            recall_all += recall
            accuracy_all += accuracy
        if tp!= 0:
            hit_count +=1
        # if count_1 == 0:
        #     len_query -= 1
    print("hit_count",hit_count)
    print("len_query",len_query)
    for item in D_pair_:
        for score in item:
            score_pair_.extend(score)
    # print("score_pair",len(score_pair_))
    # print("label_pair",len(label_pair_))
    print(f"top_{top_k},recall_ave_model: {float(recall_all/count)}")
    print(f"top_{top_k},precision_ave_model: {float(precision_all/count)}")
    mAP = average_precision_score(label_pair_, score_pair_, average='macro')
    print(f"top_{top_k},mAP_model:", mAP)
    #print(f"accuracy_ave_m3e: {float(accuracy_all/count)}")
    print(f"top_{top_k},hit_rate_ave_model:{float(hit_count/len_query)}")
    end_time = time.time()
    elapsed_time = end_time - start_time
    # print("text_list",text_list)
    print(f"Time spent: {elapsed_time:.5f} seconds")

top_k 2
hit_count 16
len_query 29
top_2,recall_ave_model: 0.10391294903083197
top_2,precision_ave_model: 0.54
top_2,mAP_model: 0.6070033459750657
top_2,hit_rate_ave_model:0.5517241379310345
Time spent: 2.24649 seconds
top_k 4
hit_count 23
len_query 29
top_4,recall_ave_model: 0.24352186875763462
top_4,precision_ave_model: 0.56
top_4,mAP_model: 0.6069656504625862
top_4,hit_rate_ave_model:0.7931034482758621
Time spent: 2.20874 seconds
top_k 6
hit_count 26
len_query 29
top_6,recall_ave_model: 0.38595862731227615
top_6,precision_ave_model: 0.58
top_6,mAP_model: 0.5902823858085461
top_6,hit_rate_ave_model:0.896551724137931
Time spent: 2.29291 seconds
top_k 8
hit_count 27
len_query 29
top_8,recall_ave_model: 0.4582630737346054
top_8,precision_ave_model: 0.565
top_8,mAP_model: 0.581921607235468
top_8,hit_rate_ave_model:0.9310344827586207
Time spent: 2.21410 seconds
top_k 10
hit_count 27
len_query 29
top_10,recall_ave_model: 0.5331104867688669
top_10,precision_ave_model: 0.54
top_10,mAP_model: 

In [13]:
#model ->计算mAP
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score,precision_recall_curve
length = len(label_pair_)
# 计算不同阈值下的精确率和召回率
precision, recall, thresholds = precision_recall_curve(label_pair_, score_pair_)
print("precision",sum(precision)/length)
print("recall",sum(recall)/length)
print("thresholds",sum(thresholds)/length)
# 计算每个阈值下的平均精度均值
# average_precision = average_precision_score(true_labels, predicted_probs)
mAP = average_precision_score(label_pair_, score_pair_, average='macro')
print("mAP_glm1:", mAP)

# 绘制曲线图
plt.plot(recall, precision, color='b', label=f'mAP={mAP:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('ChatGLM 1_mAP Curve')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_47688/279318437.py:6 in <module>                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_47688/279318437.py'                         │
│                                                                                                  │
│ /home/pai/envs/kb_chat/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:878 in            │
│ precision_recall_curve                                                                           │
│                                                                                                  │
│    875 │   >>> thresholds                                                                        │
│    876 │   array([0.1 , 0.35, 0.4 , 0.8 ])                                                       │
│    877 │   """                                                                                   │
│ ❱  878 │   fps, tps, thresholds = _binary_clf_curve(                                             │
│    879 │   │   y_true, probas_pred, pos_label=pos_label, sample_weight=sample_weight             │
│    880 │   )                                                                                     │
│    881                                                                                           │
│                                                                                                  │
│ /home/pai/envs/kb_chat/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:766 in            │
│ _binary_clf_curve                                                                                │
│                                                                                                  │
│    763 │   │   y_score = y_score[nonzero_weight_mask]                                            │
│    764 │   │   sample_weight = sample_weight[nonzero_weight_mask]                                │
│    765 │                                                                                         │
│ ❱  766 │   pos_label = _check_pos_label_consistency(pos_label, y_true)                           │
│    767 │                                                                                         │
│    768 │   # make y_true a boolean vector                                                        │
│    769 │   y_true = y_true == pos_label                                                          │
│                                                                                                  │
│ /home/pai/envs/kb_chat/lib/python3.9/site-packages/sklearn/metrics/_base.py:243 in               │
│ _check_pos_label_consistency                                                                     │
│                                                                                                  │
│   240 │   │   )                                                                                  │
│   241 │   ):                                                                                     │
│   242 │   │   classes_repr = ", ".join(repr(c) for c in classes)                                 │
│ ❱ 243 │   │   raise ValueError(                                                                  │
│   244 │   │   │   f"y_true takes value in {{{classes_repr}}} and pos_label is not "              │
│   245 │   │   │   "specified: either make y_true take value in {0, 1} or "                       │
│   246 │   │   │   "{-1, 1} or pass pos_label explicitly."                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: y_true takes value in {} and pos_label is not specified: either make y_true take value in {0, 1} or 
{-1, 1} or pass pos_label explicitly.

In [14]:
#存储检索结果
top_k = 6
# text_list = []
result = {}  # 创建空字典    
for i,batch_data in enumerate(test_data):
     query = test_data[batch_data]["query"]
     # data = test_data[batch_data]["content"]
     # label = test_data[batch_data]["label"]
     query_feat_ = model(query) # 得到model query embed
     query_feat_ = query_feat_.cpu()
     query_feat_ = np.array([query_feat_.detach()])
     faiss.normalize_L2(query_feat_)
     D_, I_ = index_.search(query_feat_, top_k) 
     text_list = []
     for text in I_[0]:
          text_list.append(testing_text[text])
     # print(text_list)
          # content = testing_text[text]
     result[i] = {
          "query": query,
          "content": text_list,
          "score": D_
     }  
     result[i]['score'] = result[i]['score'].tolist()
# print(result)
     

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_47688/1965691296.py:9 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_47688/1965691296.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [15]:
import json
# 将字典转换为 JSON 字符串
json_data = json.dumps(result,ensure_ascii=False)
# 将 JSON 字符串保存到本地文件
file_path = "result.json"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(json_data)

In [16]:
#单个查询

query = "急性肠胃炎吃什么药？"
top_k = 5
start_time = time.time()
query_feat = np.array([encoder.encode(query)])   
faiss.normalize_L2(query_feat)
D, I = index.search(query_feat, top_k) 
text_list = []
for text in I[0]:
    # text_list.append(testing_text[text])
    print(testing_text[text])
#print(D)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time spent: {elapsed_time:.2f} seconds")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_47688/3711918685.py:6 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_47688/3711918685.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'encoder' is not defined

In [17]:
#单个查询
# query = "急性肠胃炎吃什么药？"
# top_k = 2
start_time = time.time()
query_ = model(query) # 得到model query embed
query_= query_.cpu()
query_ = np.array([query_.detach()])
faiss.normalize_L2(query_)
D_, I_ = index_.search(query_, top_k) 
text_list = []
for text in I_[0]:
    # text_list.append(testing_text[text])
    print(testing_text[text])
# print(D_)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Time spent: {elapsed_time:.2f} seconds")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_47688/2627966416.py:5 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_47688/2627966416.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [18]:
# #把m3e和model检索出的结果的label设置为1，其余的设置为0
# # top_k = 10
# start_time = time.time()
# for e,batch_data in enumerate(testing_dataloader):
#     count=0
#     content_list_test = []
#     query = batch_data[1][0]
#     data, label = batch_data[2:]
#     # print(data)
#     # query_feat = np.array([encoder.encode(query)])   
#     # faiss.normalize_L2(query_feat)
#     # D, I = index.search(query_feat, top_k) 
#     # print(I[0])
#     query_ = model(query) # 得到model query embed
#     query_= query_.cpu()
#     query_ = np.array([query_.detach()])
#     faiss.normalize_L2(query_)
#     for content in data:
#         # print(content)
#         content_vectors_ = model(content)
#         content_vectors_ = content_vectors_.cpu()
#         content_vectors_ = content_vectors_.detach().numpy()
#         content_list_test.append(content_vectors_)
#     #print(len(content_list_test))
#     vectors_test = np.array(content_list_test,dtype=np.float32)
#     vectors_test = vectors_test.reshape(len(content_list_test),dim)
#     #print(vectors_test.shape)
#     # 选择faiss index
#     faiss.normalize_L2(vectors_test)
#     index_test= faiss.IndexFlatIP(dim)
#     # 数据插入
#     # 训练索引获取重构能力
#     index_test.train(vectors_test)
#     index_test.add(vectors_test)
#     D_, I_ = index_test.search(query_, len(content_list_test)) 
#     print(I_[0])
#     for idx,value in enumerate(I_[0]):
#         if idx < int(len(content_list_test)/2) and labels_recall[e][value]==1:
#             labels_recall[e][value]=1
#         else:
#             labels_recall[e][value]=0
#     print(labels_recall[e])                
#     #print("count",count)
     